In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_ds = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_ds = train_ds.flow_from_directory('training_set/training_set', target_size=(64, 64), 
                                        batch_size=32, class_mode='binary')

Found 8005 images belonging to 2 classes.


In [3]:
test_ds = ImageDataGenerator(rescale = 1./255)
test_ds = test_ds.flow_from_directory('test_set/test_set/', target_size=(64, 64), batch_size=32, 
                                      class_mode='binary')

Found 2023 images belonging to 2 classes.


In [4]:
# groups a linear stack of layers into a tf.keras.Model.
model = tf.keras.models.Sequential()

# activation='relu' - Застосовує функцію активації випрямленої linear unit.
# input_shape=[64, 64, 3] (для картинки розміром 64х64) / задаєм цей аргумент, оскільки це перший шар моделі.
# Conv2d - шар 2D згортки (наприклад, просторова згортка над зображеннями).
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

#MaxPool2D - максимальна операція об’єднання для двовимірних просторових даних.
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [5]:
model.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [6]:
# Flatten - Згладжує введення. Не впливає на batch_size.
# Batch_size визначає кількість зразків, які будуть поширюватися через мережу.
# Якщо у нас наприклад є 1050 зразків для навчання, ми візьмемо batch_size 100 і будемо брати з 0 - 100 і навчати, потім 101 - 200, ...
# І в кінці залишаться 50 і цьому може бути проблема. Тут і допоможе batch_size=50. 
model.add(tf.keras.layers.Flatten())

In [7]:
# Dence - Просто ваш звичайний щільно зв’язаний шар NN.
# units - Ціле додатне число, розмірність вихідного простору.
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [8]:
# activation='sigmoid' - Сигмовидна функція активації, sigmoid(x) = 1 / (1 + exp(-x)).
# Для малих значень (<-5) sigmoidповертає значення, близьке до нуля,
# а для великих значень (>5) результат функції наближається до 1.
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [9]:
#The compile() method: specifying a loss, metrics, and an optimizer
#To train a model with fit(), you need to specify a loss function, an optimizer, and optionally, some metrics to monitor.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
epochs = 20
# callbacks (save the model at each epoch)
callbacks = [
    keras.callbacks.ModelCheckpoint("checkpoints/model_at_{epoch}.h5"),
]
model.fit(x=train_ds, validation_data=test_ds, epochs=epochs, callbacks=callbacks)

Epoch 1/20
251/251 [==============================] - 49s 193ms/step - loss: 0.6665 - accuracy: 0.5860 - val_loss: 0.6264 - val_accuracy: 0.6466
Epoch 2/20
251/251 [==============================] - 55s 217ms/step - loss: 0.6047 - accuracy: 0.6762 - val_loss: 0.5875 - val_accuracy: 0.7098
Epoch 3/20
251/251 [==============================] - 54s 214ms/step - loss: 0.5615 - accuracy: 0.7109 - val_loss: 0.5396 - val_accuracy: 0.7370
Epoch 4/20
251/251 [==============================] - 54s 215ms/step - loss: 0.5248 - accuracy: 0.7373 - val_loss: 0.5041 - val_accuracy: 0.7593
Epoch 5/20
251/251 [==============================] - 54s 215ms/step - loss: 0.5023 - accuracy: 0.7502 - val_loss: 0.4841 - val_accuracy: 0.7657
Epoch 6/20
251/251 [==============================] - 47s 187ms/step - loss: 0.4760 - accuracy: 0.7725 - val_loss: 0.4830 - val_accuracy: 0.7751
Epoch 7/20
251/251 [==============================] - 47s 188ms/step - loss: 0.4645 - accuracy: 0.7780 - val_loss: 0.5487 - val_ac

In [99]:
for i in range(1, 9):
    tf.keras.utils.load_img(f'cat_dog_to_predict/{i}.jpg')

In [93]:
def predic_animal(filename, model, i):
    test_img = tf.keras.utils.load_img(filename, target_size = (64, 64))
    img = tf.keras.utils.img_to_array(test_img)
    img = np.expand_dims(img, axis = 0)
    
    res = model.predict(img)
    #train_ds.class_indices # ?
    if res[0][0] == 1:
        print(f"Image {i} - Dog")
    else:
        print(f"Image {i} - Cat")


In [95]:
for i in range(1, 9):
    predic_animal(f'cat_dog_to_predict/{i}.jpg', model, i)

1/1 [==============================] - 0s 24ms/step
Image 1 - Dog
1/1 [==============================] - 0s 23ms/step
Image 2 - Dog
1/1 [==============================] - 0s 25ms/step
Image 3 - Dog
1/1 [==============================] - 0s 34ms/step
Image 4 - Dog
1/1 [==============================] - 0s 12ms/step
Image 5 - Cat
1/1 [==============================] - 0s 24ms/step
Image 6 - Dog
1/1 [==============================] - 0s 23ms/step
Image 7 - Cat
1/1 [==============================] - 0s 15ms/step
Image 8 - Dog
